In [26]:
import time
from numpy.random import choice
import random
import time
from inspect import getfullargspec

In [3]:
""" generator zwracajacy czas wykonania funkcji"""
def time_mes(fn):
    def modified_fn(*args):
        start = time.time() # zaczecie liczenia czasu
        fn(*args)
        return time.time() - start # zwrocenie roznicy czasu 
    return modified_fn

"""funkcja liczaca iloczyn trzech liczb"""
@time_mes
def x(a,b,c):
    time.sleep(1)
    return a*b*c

print(x(5,6,7))

1.0003259181976318


In [9]:
""" dodanie liscia do drzewa """
def add_leaf(i, tree):
    k = 0
    for j,x in enumerate(tree):
        if x == None: # jeśli jest liściem pustym
            # jesli True to dodam nowego w miejsce tego pustego liścia
            if random.choice([True,False]): 
                tree[j] = [str(i), None, None]
                k = i+1
        # jeśli liść jest listą to szukam tam wartości None, aby spróbować dodajć liść
        elif isinstance(x, list): 
            tree[j], k = add_leaf(i,x)
        # k > i -> liść został dodany
        if k > i:
            return tree, k
    return tree, i

""" zwraca wysokość drzewa """
def height(tree, h=0, h_max = 0):
    # idę jak najniżej się da na każdej gałęzi
    for leaf in tree:
        if isinstance(leaf, list):
            h, h_max = height(leaf, h+1, h_max)

    # zmieniam h_max jeśli znalazłem dłużą gałąź        
    if h_max < h:
        h_max = h
    return h-1, h_max

""" tworzenie drzewa o podanej wysokości n """
def gen_tree(n):
    tree = ["0",None, None]
    i = 1
    k = 0
    while max(height(tree)) < n:
        # k > i -> liść został dodany
        while k <= i:
            tree, k = add_leaf(i ,tree)
        i = k
    return tree

# generator dfs chodzący po drzewie tree
def dfs(tree):
    for leaf in tree:
        if leaf == None:
            continue
        elif isinstance(leaf,str):
            yield leaf
        elif isinstance(leaf,list):
            for x in dfs(leaf):
                yield x

# generator bfs chodzący po drzewie tree
def bfs(tree):
    q = []
    for leaf in tree:
        if leaf == None:
            continue
        if leaf[0] == "0":
            yield leaf[0]
        if isinstance(leaf, list):
            yield leaf[0]
            q.append(leaf)
    
    for leaf in q:
       for x in bfs(leaf):
           yield x

tree = gen_tree(5)
print(tree)
print(list(dfs(tree)))
print(list(bfs(tree)))


['0', ['1', ['2', ['5', ['7', None, None], ['6', None, ['8', None, None]]], ['3', ['4', None, None], None]], None], None]
['0', '1', '2', '5', '7', '6', '8', '3', '4']
['0', '1', '2', '5', '3', '7', '6', '8', '4']


In [1]:
""" klasa opisująca obiekt drzewa """
class Node(object):


    """ konstruktor klasy Node """
    def __init__(self, father, name, height):
        self.father = father
        self.name = name
        self.childs = []
        self.height = height

    """ dodanie dziecka, do któregoś z node'ów """
    def add_child(self, child_name):
        # 30% na dodanie dziecka do tego node'a
        if choice([True,False],1,[3,7]):
            new_child = Node(self.name, child_name, self.height + 1)
            self.childs.append(new_child)
            return True
        return False

    """ zwrócenie dzieci node'a """
    def get_childs(self):
        return self.childs

    """ zwrócenie wysokości na której się znajduje node """
    def get_height(self):
        return self.height
    
    """ zwrócenie nazwy (numeru) node'a """
    def get_name(self):
        return self.name

""" wstawienie nowego liścia """
def insert_leaf(i, tree):
    k = 0
    if tree.add_child(str(i)):
        return i+1
    else:
        # jeśli się nie udało wstawić do node'a
        # to próba wstawienia, do którego z dzieci
        for child in tree.get_childs():
            k = insert_leaf(i, child)
            if k > i:
                return k
    return i

""" zwrócenie wysokości drzewa """
def height(tree, h=0):
    # znalezienie najwiekszej wartości get_height() i jej zwrócenie
    if h < tree.get_height():
        h = tree.get_height()
    for leaf in tree.get_childs():
        h = height(leaf,h)
    return h

""" generowanie drzewa wysokości n """
def gen_tree(n):
    tree = Node(None, "0" ,0)
    i = 1
    k = 0
    while height(tree) < n:
        while k <= i:
            k = insert_leaf(i, tree)
        i = k
    return tree

""" generator dfs chodzący po tree """
def dfs(tree):
    yield tree.get_name()
    for child in tree.get_childs():
        for x in dfs(child):
            yield x
    return

""" generator bfs chodzący po tree """
def bfs(tree):
    q = []
    if tree.get_name() == "0":
        yield tree.get_name()
    
    for x in tree.get_childs():
        yield x.get_name()
        q.append(x)
    
    for x in q:
        for y in bfs(x):
            yield y

    return

""" wyświetlenie drzewa w sposób Node : [Dzieci] """
def display_tree(tree):
    if len(tree.get_childs()) != 0:
        print("Father: ",tree.get_name(),end=' ')
        print("Children: ",end='')
        for x in tree.get_childs():
            print(x.get_name(),end=' ')
        print("\n")
        for x in tree.get_childs():
            display_tree(x)

tree = gen_tree(5)
display_tree(tree)
print(list(dfs(tree)))
print(list(bfs(tree)))

IndentationError: expected an indented block (<ipython-input-1-2f0c2f2a66aa>, line 2)

In [47]:
class Overload(object):
""" klasa overload zwracająca funkcję o odpowiedniej ilości argumentów """

    functions = []

    """ konstruktor klasy """
    def __init__(self,fn):
        # dodanie funkcji do listy functions
        self.functions.append(fn)

    """ wywołanie odpowiedniej funkcji z listy """
    def __call__(self,*args):
        for fn in self.functions:
            try:
                return fn(*args)
            except:
                continue

""" funckja zwracająca iloczyn trzech liczb"""
@Overload
def x(a,b,c):
    return a*b*c

""" funkcja zwracająca iloczyn dwóch liczb """
@Overload
def x(a,b):
    return a*b

print(x(5,6,7))
print(x(5,6))

210
30
